In [1]:
import numpy as NP
import os 
import math
from pathlib import Path
import matplotlib.pyplot as PLT

In [2]:
from sklearn.utils import Bunch
from sklearn.model_selection import train_test_split

In [3]:
import skimage
from skimage.io import imread
from skimage.transform import resize

In [10]:
def load_image_files(container_path, dimension=(100,100)):
    folders = [dir_path for dir_path in Path(container_path).iterdir()] 
    categories = [fo.name for fo in folders]
    
    descr = "Alphacorona & Coronaviridae"
    images = []
    flat_data = []
    target = []
    
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = skimage.io.imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = NP.array(flat_data)
    target = NP.array(target)
    images = NP.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [11]:
a = load_image_files("C:\\Users\\DELL\\Desktop\\AI - Amrita\\Datasets\\CGR_Project")

In [12]:
dir(a)

['DESCR', 'data', 'images', 'target', 'target_names']

In [16]:
print(NP.shape(a.target))

print(NP.shape(a.target_names))

print(NP.shape(a.data))

print(NP.shape(a.images))

(263,)
(2,)
(263, 40000)
(263, 100, 100, 4)


In [25]:
train_set_x_orig, test_set_x_orig, tr_set_y, tes_set_y =  train_test_split(a.images, a.target, test_size = 0.25)

In [26]:
print(NP.shape(train_set_x_orig))

print(NP.shape(test_set_x_orig))

print(NP.shape(tr_set_y))

print(NP.shape(tes_set_y))

num_px = train_set_x_orig.shape[1]
print(num_px)

(197, 100, 100, 4)
(66, 100, 100, 4)
(197,)
(66,)
100


## Train, Test sets and labels READY!!

In [28]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
print(NP.shape(train_set_x_flatten))

train_set_y = train_set_y.reshape((1,len(tr_set_y)))
print(NP.shape(train_set_y))

test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
print(NP.shape(test_set_x_flatten))

test_set_y = tes_set_y.reshape((1,len(tes_set_y)))
print(NP.shape(test_set_y))

(40000, 197)
(1, 197)
(40000, 66)
(1, 66)


In [29]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

In [30]:
print(NP.shape(train_set_x))
print(NP.shape(test_set_x))

(40000, 197)
(40000, 66)


In [34]:
def sigmoid(z):
    return 1/(1 + NP.exp(-z))

In [69]:
def initialize_with_zeros(dim):
    
    w = NP.zeros((dim, 1))
    b = 0

    assert (w.shape == (dim, 1))
    assert (isinstance(b, float) or isinstance(b, int))

    return w, b


In [37]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]

    A = sigmoid(NP.dot(w.T, X) + b)  # compute activation
    cost = (1. / m) * (-NP.dot(Y, NP.log(A).T) - NP.dot(1 - Y, NP.log(1 - A).T))  # compute cost

    # BACKWARD PROPAGATION (TO FIND GRAD)

    dw = 1. / m * NP.dot(X, (A - Y).T)
    db = 1. / m * NP.sum(A - Y)

    assert (dw.shape == w.shape)
    assert (db.dtype == float)
    cost = NP.squeeze(cost)
    assert (cost.shape == ())

    grads = {"dw": dw, "db": db}

    return grads, cost


In [45]:
X = NP.random.randint(1,5,size=(2*2*4,10))
Y = NP.random.randint(0,2,size=(    1,10))

w = NP.random.rand(2*2*4,1)

In [46]:
grads, cost = propagate(w, 0, X, Y)

In [53]:
print(cost)
print(NP.shape(grads["dw"]))
print(grads["db"])

6.563157474984694
(16, 1)
0.29999998016861606


In [54]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    
    costs = []

    for i in range(num_iterations):

        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)

        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]

        # update rule
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the costs
        if i % 100 == 0:
            costs.append(cost)

        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    params = {"w": w,
              "b": b}

    grads = {"dw": dw,
             "db": db}

    return params, grads, costs


In [56]:
param, grads_new, costs = optimize(w, 0, X, Y, 400, 0.005, print_cost=True)

Cost after iteration 0: 6.563157
Cost after iteration 100: 1.542720
Cost after iteration 200: 0.361785
Cost after iteration 300: 0.267294


In [58]:
print(param["b"])
print(NP.shape(param["w"]))

print(grads_new["db"])
print(NP.shape(grads_new["dw"]))

print(NP.shape(costs))

-0.17648839058218646
(16, 1)
-0.010997205889646096
(16, 1)
(4,)


In [74]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = NP.zeros((1, m))
    w = w.reshape(X.shape[0], 1)

    # Compute vector "A" predicting the probabilities of a cat being present in the picture

    A = sigmoid(NP.dot(w.T, X) + b)
    print("\n", A)

    for i in range(A.shape[1]):

        # Convert probabilities A[0,i] to actual predictions p[0,i]

        for i in range(0, A.shape[1]):
            
            if A[0, i] > 0.5:
                Y_prediction[0, i] = 1
            else:
                Y_prediction[0, i] = 0

    assert (Y_prediction.shape == (1, m))

    return Y_prediction

In [66]:
predict(w,0,X)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [75]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=400, learning_rate=0.005, print_cost=False):
    

    # initialize parameters with zeros
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]

    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - NP.mean(NP.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - NP.mean(NP.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d

In [76]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 400, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.634010
Cost after iteration 200: 0.592619
Cost after iteration 300: 0.563402

 [[0.65551684 0.6570018  0.65560386 0.65743496 0.6569533  0.65726592
  0.65727461 0.65678569 0.6578536  0.65507739 0.65699689 0.65666875
  0.65618239 0.65629994 0.65696871 0.65551695 0.65618583 0.6569723
  0.65630925 0.65519484 0.65544266 0.65614045 0.65639111 0.65497214
  0.65614436 0.65711421 0.65537938 0.65700182 0.65499109 0.65531964
  0.65657352 0.65716746 0.65579701 0.65639111 0.65603441 0.65716746
  0.65697654 0.65715162 0.6578536  0.65554901 0.65583029 0.65713441
  0.65572838 0.65613842 0.65609855 0.65624257 0.65622167 0.65710593
  0.6562709  0.65641602 0.65706888 0.65618732 0.65555014 0.65624331
  0.65768113 0.65542876 0.65709347 0.65696828 0.65678247 0.65554743
  0.65694543 0.65715162 0.65677245 0.65693732 0.65698482 0.65483047]]

 [[0.65701095 0.65712571 0.65551105 0.65620431 0.65554727 0.6549647
  0.65587581 0.65651235 0.65456802 0.65621

In [80]:
NP.mean(d["Y_prediction_test"] - test_set_y)

0.13636363636363635